In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import torch.utils.data as utils
import librosa
import soundfile as sf
import time
import os

In [2]:
sampleSize=50000
sample_rate=16000
quantization_channels=256
#dilations=[2**i for i in range(8)]*20
#"residualDim=32
dilations=[2**i for i in range(10)]*5
residualDim=96
skipDim=256
filterSize=3
pad = np.sum(dilations)
shapeoftest=190500
lossrecord=[]
pad

5115

In [3]:
#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [4]:
use_cuda = torch.cuda.is_available()
torch.manual_seed(1)
device = torch.device("cuda" if use_cuda else "cpu")
#device = 'cpu'
torch.set_default_tensor_type('torch.cuda.FloatTensor')
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [5]:
def mu_law_encode(audio, quantization_channels=quantization_channels):
    '''Quantizes waveform amplitudes.'''
    mu = (quantization_channels - 1)*1.0
    # Perform mu-law companding transformation (ITU-T, 1988).
    # Minimum operation is here to deal with rare large amplitudes caused
    # by resampling.
    safe_audio_abs = np.minimum(np.abs(audio), 1.0)
    magnitude = np.log1p(mu * safe_audio_abs) / np.log1p(mu)
    signal = np.sign(audio) * magnitude
    # Quantize signal to the specified number of levels.
    #if(forX):return signal
    return ((signal + 1) / 2 * mu + 0.5).astype(int)
def mu_law_decode(output, quantization_channels=quantization_channels):
    '''Recovers waveform from quantized values.'''
    mu = quantization_channels - 1
    # Map values back to [-1, 1].
    signal = 2 * ((output*1.0) / mu) - 1
    # Perform inverse of mu-law transformation.
    magnitude = (1 / mu) * ((1 + mu)**np.abs(signal) - 1)
    return np.sign(signal) * magnitude

In [6]:
def readAudio(name):
    audio0, samplerate = sf.read(name, dtype='float32')
    return librosa.resample(audio0.T, samplerate, sample_rate).reshape(-1)
p=['./vsCorpus/origin_mix.wav','./vsCorpus/origin_vocal.wav',
   './vsCorpus/origin_mix.wav','./vsCorpus/origin_vocal.wav','./vsCorpus/pred_mix.wav']
xtrain,ytrain,xval,yval,xtest=readAudio(p[0]),readAudio(p[1]),readAudio(p[2]),readAudio(p[3]),readAudio(p[4])
assert((xtrain==xval).all())
assert((ytrain==yval).all())
assert((xtrain != ytrain).any())

In [7]:
xmean,xstd = xtrain.mean(),xtrain.std()
xtrain=(xtrain-xmean)/xstd
xval=(xval-xmean)/xstd
xtest=(xtest-xmean)/xstd
ytrain,yval=mu_law_encode(ytrain),mu_law_encode(yval)

In [8]:
xtrain,ytrain=xtrain[:xtest.shape[0]],ytrain[:xtest.shape[0]]
xval,yval=xval[:xtest.shape[0]],yval[:xtest.shape[0]]
xtrain=np.pad(xtrain, (pad, pad), 'constant')
xval=np.pad(xval, (pad, pad), 'constant')
xtest=np.pad(xtest, (pad, pad), 'constant')
yval=np.pad(yval, (pad, pad), 'constant')
ytrain=np.pad(ytrain, (pad, pad), 'constant')

In [9]:
#xtrain,ytrain,xval,yval=xtrain[:-sampleSize],ytrain[:-sampleSize],xval[-sampleSize:],yval[-sampleSize:]
#xtrain,ytrain,xval,yval=xtrain[:-sampleSize],ytrain[:-sampleSize],xval[:sampleSize],yval[:sampleSize]
xtrain,xval,xtest=xtrain.reshape(1,1,-1),xval.reshape(1,1,-1),xtest.reshape(1,1,-1)
ytrain,yval=ytrain.reshape(1,-1),yval.reshape(1,-1)

In [10]:
xtrain,ytrain,xval,yval,xtest = torch.from_numpy(xtrain).type(torch.float32),\
                                torch.from_numpy(ytrain).type(torch.LongTensor),\
                                torch.from_numpy(xval).type(torch.float32),\
                                torch.from_numpy(yval).type(torch.LongTensor),\
                                torch.from_numpy(xtest).type(torch.float32)

In [11]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        sd,qd,rd = skipDim,quantization_channels,residualDim
        self.causal = nn.Conv1d(in_channels=1,out_channels=rd,kernel_size=3,padding=1)
        self.layer=dict()
        for i, d in enumerate(dilations):
            self.layer['tanh'+str(i)] = nn.Conv1d(in_channels=rd,out_channels=rd,kernel_size=3,padding=0,dilation=d)
            self.layer['sigmoid'+str(i)] = nn.Conv1d(in_channels=rd,out_channels=rd,kernel_size=3,padding=0,dilation=d)
            self.layer['skip'+str(i)] = nn.Conv1d(in_channels=rd,out_channels=sd,kernel_size=1,padding=0)
            self.layer['dense'+str(i)] = nn.Conv1d(in_channels=rd,out_channels=rd,kernel_size=1,padding=0)
        self.post1 = nn.Conv1d(in_channels=sd,out_channels=sd,kernel_size=1,padding=0)
        self.post2 = nn.Conv1d(in_channels=sd,out_channels=qd,kernel_size=1,padding=0)
        self.tanh,self.sigmoid = nn.Tanh(),nn.Sigmoid()

    def forward(self, x):
        finallen = x.shape[-1]-2*pad
        x = self.causal(x)
        #print('x.shape',x.shape)
        skip_connections = torch.zeros([1,skipDim,finallen],dtype=torch.float32,device=device)
        for i, dilation in enumerate(dilations):
            xinput = x.clone()[:,:,dilation:-dilation]
            x1 = self.tanh(self.layer['tanh'+str(i)](x))
            #print('tanh.shape',x1.shape)
            x2 = self.sigmoid(self.layer['sigmoid'+str(i)](x))
            #print('sigmoid.shape',x2.shape)
            x = x1*x2
            #print('multi',x3.shape)
            cutlen = (x.shape[-1] - finallen)//2
            skip_connections += (self.layer['skip'+str(i)](x)).narrow(2,int(cutlen),int(finallen))
            #cur =self.layer['skip'+str(i)](x3)
            x = self.layer['dense'+str(i)](x)
            #print('dense.shape',x.shape)
            x += xinput
        x = self.post2(F.relu(self.post1(F.relu(skip_connections))))
        return x

model = Net().cuda()
criterion = nn.CrossEntropyLoss().cuda()
#optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)
optimizer = optim.Adam(model.parameters(),weight_decay=1e-5)

In [12]:
#torch.sum(c,dim=0,keepdim=True)

In [13]:
#model = torch.load('torchmodel0.681600')

In [ ]:
def val():
    model.eval()
    startval_time = time.time()
    with torch.no_grad():
        #data, target = xval.to(device), yval.to(device)
        data, target = xtrain[:,:,0:2*pad+shapeoftest].to(device), ytrain[:,pad:shapeoftest+pad].to(device)
        output = model(data)
        #print(output.shape)
        #print(output[:,:,:10])
        pred = output.max(1, keepdim=True)[1]
        correct = pred.eq(target.view_as(pred)).sum().item() / pred.shape[-1]
        val_loss = criterion(output, target).item()
    print(correct,'accurate')
    print('\nval set:loss{:.4f}:, ({:.3f} sec/step)\n'.format(val_loss,time.time()-startval_time))

def test():
    model.eval()
    startval_time = time.time()
    with torch.no_grad():
        output = model(xtest.to(device))
        pred = output.max(1, keepdim=True)[1].cpu().numpy().reshape(-1)
        ans = mu_law_decode(pred)
        sf.write('./vsCorpus/resultxte.wav', ans, sample_rate)
        
        #output = model(xtrain[:,:,:sampleSize].to(device))
        output = model(xtrain[:,:,0:2*pad+shapeoftest].to(device))
        pred = output.max(1, keepdim=True)[1].cpu().numpy().reshape(-1)
        ans = mu_law_decode(pred)
        sf.write('./vsCorpus/resultxtr.wav', ans, sample_rate)
        print('stored done\n')
    
def train(epoch):
    model.train()
    #idx = np.arange(xtrain.shape[-1] - 2 * sampleSize,1000)
    #176000
    idx = np.arange(pad,shapeoftest+pad-sampleSize,1000)
    np.random.shuffle(idx)
    for i, ind in enumerate(idx):
        start_time = time.time()
        data, target = xtrain[:,:,ind-pad:ind+sampleSize+pad].to(device), ytrain[:,ind:ind+sampleSize].to(device)
        output = model(data)
        #print(output.shape)
        loss = criterion(output, target)
        lossrecord.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print('Train Epoch: {} [{}/{} ({:.0f}%)] Loss:{:.6f}: , ({:.3f} sec/step)'.format(
                epoch, i, len(idx),100. * i / len(idx), loss.item(),time.time() - start_time))
        if i % 100 == 0:
            val()
            test()
            torch.save(model, 'padonlyonside')

In [ ]:
for epoch in range(100000):
    train(epoch)
    test()

Train Epoch: 0 [0/141 (0%)] Loss:5.557175: , (2.242 sec/step)
0.005889763779527559 accurate

val set:loss5.5286:, (2.553 sec/step)

stored done



/home/coder.chenshicheng/anaconda3/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Train Epoch: 0 [1/141 (1%)] Loss:5.524753: , (2.240 sec/step)
Train Epoch: 0 [2/141 (1%)] Loss:5.499656: , (2.240 sec/step)
Train Epoch: 0 [3/141 (2%)] Loss:5.471137: , (2.242 sec/step)
Train Epoch: 0 [4/141 (3%)] Loss:5.434199: , (2.243 sec/step)
Train Epoch: 0 [5/141 (4%)] Loss:5.412224: , (2.249 sec/step)
Train Epoch: 0 [6/141 (4%)] Loss:5.351348: , (2.248 sec/step)
Train Epoch: 0 [7/141 (5%)] Loss:5.339950: , (2.249 sec/step)
Train Epoch: 0 [8/141 (6%)] Loss:5.308004: , (2.249 sec/step)
Train Epoch: 0 [9/141 (6%)] Loss:5.299894: , (2.250 sec/step)
Train Epoch: 0 [10/141 (7%)] Loss:5.318152: , (2.249 sec/step)
Train Epoch: 0 [11/141 (8%)] Loss:5.258930: , (2.251 sec/step)
Train Epoch: 0 [12/141 (9%)] Loss:5.207211: , (2.250 sec/step)
Train Epoch: 0 [13/141 (9%)] Loss:5.192152: , (2.251 sec/step)
Train Epoch: 0 [14/141 (10%)] Loss:5.154831: , (2.251 sec/step)
Train Epoch: 0 [15/141 (11%)] Loss:5.128887: , (2.251 sec/step)
Train Epoch: 0 [16/141 (11%)] Loss:5.123178: , (2.252 sec/step

Train Epoch: 0 [128/141 (91%)] Loss:4.374453: , (2.263 sec/step)
Train Epoch: 0 [129/141 (91%)] Loss:4.399303: , (2.263 sec/step)
Train Epoch: 0 [130/141 (92%)] Loss:4.580882: , (2.263 sec/step)
Train Epoch: 0 [131/141 (93%)] Loss:4.530675: , (2.263 sec/step)
Train Epoch: 0 [132/141 (94%)] Loss:4.489211: , (2.263 sec/step)
Train Epoch: 0 [133/141 (94%)] Loss:4.507203: , (2.262 sec/step)
Train Epoch: 0 [134/141 (95%)] Loss:4.468691: , (2.263 sec/step)
Train Epoch: 0 [135/141 (96%)] Loss:4.544922: , (2.263 sec/step)
Train Epoch: 0 [136/141 (96%)] Loss:4.430676: , (2.262 sec/step)
Train Epoch: 0 [137/141 (97%)] Loss:4.500751: , (2.263 sec/step)
Train Epoch: 0 [138/141 (98%)] Loss:4.374337: , (2.263 sec/step)
Train Epoch: 0 [139/141 (99%)] Loss:4.480306: , (2.263 sec/step)
Train Epoch: 0 [140/141 (99%)] Loss:4.433644: , (2.263 sec/step)
stored done

Train Epoch: 1 [0/141 (0%)] Loss:4.446506: , (2.263 sec/step)
0.03836745406824147 accurate

val set:loss4.4981:, (2.576 sec/step)

stored done

Train Epoch: 1 [113/141 (80%)] Loss:4.403864: , (2.265 sec/step)
Train Epoch: 1 [114/141 (81%)] Loss:4.333364: , (2.266 sec/step)
Train Epoch: 1 [115/141 (82%)] Loss:4.515179: , (2.264 sec/step)
Train Epoch: 1 [116/141 (82%)] Loss:4.337969: , (2.264 sec/step)
Train Epoch: 1 [117/141 (83%)] Loss:4.329847: , (2.266 sec/step)
Train Epoch: 1 [118/141 (84%)] Loss:4.399104: , (2.266 sec/step)
Train Epoch: 1 [119/141 (84%)] Loss:4.313577: , (2.265 sec/step)
Train Epoch: 1 [120/141 (85%)] Loss:4.285566: , (2.266 sec/step)
Train Epoch: 1 [121/141 (86%)] Loss:4.408511: , (2.265 sec/step)
Train Epoch: 1 [122/141 (87%)] Loss:4.358691: , (2.264 sec/step)
Train Epoch: 1 [123/141 (87%)] Loss:4.426523: , (2.263 sec/step)
Train Epoch: 1 [124/141 (88%)] Loss:4.307420: , (2.264 sec/step)
Train Epoch: 1 [125/141 (89%)] Loss:4.403907: , (2.264 sec/step)
Train Epoch: 1 [126/141 (89%)] Loss:4.436215: , (2.264 sec/step)
Train Epoch: 1 [127/141 (90%)] Loss:4.414941: , (2.263 sec/step)
Train Epoch: 1 [128/141 (

...

Train Epoch: 143 [119/141 (84%)] Loss:3.569233: , (2.259 sec/step)
Train Epoch: 143 [120/141 (85%)] Loss:3.224040: , (2.259 sec/step)
Train Epoch: 143 [121/141 (86%)] Loss:3.301663: , (2.259 sec/step)
Train Epoch: 143 [122/141 (87%)] Loss:3.433528: , (2.259 sec/step)
Train Epoch: 143 [123/141 (87%)] Loss:3.819435: , (2.258 sec/step)
Train Epoch: 143 [124/141 (88%)] Loss:3.550408: , (2.258 sec/step)
Train Epoch: 143 [125/141 (89%)] Loss:3.290694: , (2.258 sec/step)
Train Epoch: 143 [126/141 (89%)] Loss:3.646827: , (2.259 sec/step)
Train Epoch: 143 [127/141 (90%)] Loss:3.656034: , (2.259 sec/step)
Train Epoch: 143 [128/141 (91%)] Loss:3.596174: , (2.258 sec/step)
Train Epoch: 143 [129/141 (91%)] Loss:3.322535: , (2.258 sec/step)
Train Epoch: 143 [130/141 (92%)] Loss:3.349751: , (2.259 sec/step)
Train Epoch: 143 [131/141 (93%)] Loss:3.984724: , (2.258 sec/step)
Train Epoch: 143 [132/141 (94%)] Loss:3.618231: , (2.258 sec/step)
Train Epoch: 143 [133/141 (94%)] Loss:3.337404: , (2.258 sec/s

0.16586876640419948 accurate

val set:loss3.6719:, (2.573 sec/step)

stored done

Train Epoch: 144 [101/141 (72%)] Loss:3.539718: , (2.257 sec/step)
Train Epoch: 144 [102/141 (72%)] Loss:3.254951: , (2.257 sec/step)
Train Epoch: 144 [103/141 (73%)] Loss:3.621696: , (2.257 sec/step)
Train Epoch: 144 [104/141 (74%)] Loss:3.344516: , (2.257 sec/step)
Train Epoch: 144 [105/141 (74%)] Loss:3.749255: , (2.258 sec/step)
Train Epoch: 144 [106/141 (75%)] Loss:3.271416: , (2.258 sec/step)
Train Epoch: 144 [107/141 (76%)] Loss:3.154856: , (2.258 sec/step)
Train Epoch: 144 [108/141 (77%)] Loss:3.369565: , (2.257 sec/step)
Train Epoch: 144 [109/141 (77%)] Loss:3.289734: , (2.258 sec/step)
Train Epoch: 144 [110/141 (78%)] Loss:3.289011: , (2.257 sec/step)
Train Epoch: 144 [111/141 (79%)] Loss:3.587957: , (2.257 sec/step)
Train Epoch: 144 [112/141 (79%)] Loss:3.341205: , (2.259 sec/step)
Train Epoch: 144 [113/141 (80%)] Loss:3.110462: , (2.257 sec/step)
Train Epoch: 144 [114/141 (81%)] Loss:3.330585:

Train Epoch: 145 [82/141 (58%)] Loss:3.250471: , (2.257 sec/step)
Train Epoch: 145 [83/141 (59%)] Loss:3.376279: , (2.258 sec/step)
Train Epoch: 145 [84/141 (60%)] Loss:3.990077: , (2.257 sec/step)
Train Epoch: 145 [85/141 (60%)] Loss:3.537128: , (2.258 sec/step)
Train Epoch: 145 [86/141 (61%)] Loss:3.176862: , (2.257 sec/step)
Train Epoch: 145 [87/141 (62%)] Loss:3.189731: , (2.257 sec/step)
Train Epoch: 145 [88/141 (62%)] Loss:3.107203: , (2.258 sec/step)
Train Epoch: 145 [89/141 (63%)] Loss:3.391032: , (2.257 sec/step)
Train Epoch: 145 [90/141 (64%)] Loss:3.369539: , (2.256 sec/step)
Train Epoch: 145 [91/141 (65%)] Loss:3.582561: , (2.256 sec/step)
Train Epoch: 145 [92/141 (65%)] Loss:3.978487: , (2.256 sec/step)
Train Epoch: 145 [93/141 (66%)] Loss:3.693094: , (2.255 sec/step)
Train Epoch: 145 [94/141 (67%)] Loss:3.550992: , (2.256 sec/step)
Train Epoch: 145 [95/141 (67%)] Loss:3.572994: , (2.256 sec/step)
Train Epoch: 145 [96/141 (68%)] Loss:3.407404: , (2.257 sec/step)
Train Epoc

Train Epoch: 146 [63/141 (45%)] Loss:3.468883: , (2.257 sec/step)
Train Epoch: 146 [64/141 (45%)] Loss:3.521147: , (2.257 sec/step)
Train Epoch: 146 [65/141 (46%)] Loss:3.827781: , (2.257 sec/step)
Train Epoch: 146 [66/141 (47%)] Loss:3.393806: , (2.257 sec/step)
Train Epoch: 146 [67/141 (48%)] Loss:3.401348: , (2.256 sec/step)
Train Epoch: 146 [68/141 (48%)] Loss:3.186170: , (2.256 sec/step)
Train Epoch: 146 [69/141 (49%)] Loss:3.713789: , (2.255 sec/step)
Train Epoch: 146 [70/141 (50%)] Loss:3.293837: , (2.256 sec/step)
Train Epoch: 146 [71/141 (50%)] Loss:3.439296: , (2.257 sec/step)
Train Epoch: 146 [72/141 (51%)] Loss:3.235584: , (2.256 sec/step)
Train Epoch: 146 [73/141 (52%)] Loss:3.226889: , (2.256 sec/step)
Train Epoch: 146 [74/141 (52%)] Loss:3.120143: , (2.257 sec/step)
Train Epoch: 146 [75/141 (53%)] Loss:3.371039: , (2.256 sec/step)
Train Epoch: 146 [76/141 (54%)] Loss:3.218086: , (2.256 sec/step)
Train Epoch: 146 [77/141 (55%)] Loss:3.153269: , (2.256 sec/step)
Train Epoc

Train Epoch: 147 [44/141 (31%)] Loss:3.567291: , (2.258 sec/step)
Train Epoch: 147 [45/141 (32%)] Loss:3.237842: , (2.259 sec/step)
Train Epoch: 147 [46/141 (33%)] Loss:3.803734: , (2.259 sec/step)
Train Epoch: 147 [47/141 (33%)] Loss:3.308969: , (2.259 sec/step)
Train Epoch: 147 [48/141 (34%)] Loss:3.246883: , (2.259 sec/step)
Train Epoch: 147 [49/141 (35%)] Loss:3.163352: , (2.259 sec/step)
Train Epoch: 147 [50/141 (35%)] Loss:4.069229: , (2.259 sec/step)
Train Epoch: 147 [51/141 (36%)] Loss:3.391940: , (2.260 sec/step)
Train Epoch: 147 [52/141 (37%)] Loss:3.569657: , (2.258 sec/step)
Train Epoch: 147 [53/141 (38%)] Loss:3.414122: , (2.259 sec/step)
Train Epoch: 147 [54/141 (38%)] Loss:3.702901: , (2.258 sec/step)
Train Epoch: 147 [55/141 (39%)] Loss:3.197694: , (2.258 sec/step)
Train Epoch: 147 [56/141 (40%)] Loss:3.271258: , (2.258 sec/step)
Train Epoch: 147 [57/141 (40%)] Loss:3.280195: , (2.258 sec/step)
Train Epoch: 147 [58/141 (41%)] Loss:3.805196: , (2.257 sec/step)
Train Epoc

In [ ]:
#model = torch.load('torchmodel')

In [ ]:
#torch.save(model, 'loss23*1.+4*3.+16*2.step_9072_repeat15*2**8resu32sample50000')